In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from torch.optim import Adam
from torch.cuda.amp import GradScaler, autocast

In [19]:
def load_data(data_dir, batch_size):
    # Define transformations
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load dataset
    full_dataset = ImageFolder(root=data_dir)
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Apply transformations appropriately
    train_dataset.dataset = ImageFolder(root=data_dir, transform=transform_train)
    test_dataset.dataset = ImageFolder(root=data_dir, transform=transform_test)

    # Efficient calculation of class weights using tensor operations
    target_list = torch.tensor([y for _, y in train_dataset.dataset.samples])
    class_counts = torch.bincount(target_list[train_dataset.indices])
    class_weights = 1.0 / (class_counts.float() + 1e-6)
    sample_weights = class_weights[target_list[train_dataset.indices]]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    return train_loader, test_loader

In [20]:
def define_model(num_classes):
    # Modify the pretrained ResNet50 model
    model = resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, num_classes),
        nn.BatchNorm1d(num_classes)
    )
    return model

In [24]:
# Training the model
def train_model(model, train_loader, device, num_epochs=10):
    model.to(device)
    optimizer = Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            if i % 20 == 0:
                print(f'Epoch {epoch+1}, Step {i}, Loss: {loss.item()}')

        end_time = time.time()
        epoch_time = end_time - start_time
        print(f'Epoch {epoch+1} completed in {epoch_time:.2f} seconds')

In [25]:
# Evaluation
def evaluate_model(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')



In [26]:
def main():
    data_dir = '/content/drive/MyDrive/JPEGImages'
    batch_size = 64
    num_classes = 50
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = load_data(data_dir, batch_size)
    model = define_model(num_classes)
    train_model(model, train_loader, device)
    evaluate_model(model, test_loader, device)

if __name__ == "__main__":
    main()

Epoch 1, Step 0, Loss: 4.416748046875
Epoch 1, Step 20, Loss: 2.3235626220703125
Epoch 1, Step 40, Loss: 2.0426712036132812
Epoch 1, Step 60, Loss: 2.008150100708008
Epoch 1, Step 80, Loss: 1.734346866607666
Epoch 1, Step 100, Loss: 1.4771003723144531
Epoch 1, Step 120, Loss: 1.4335107803344727
Epoch 1 completed in 29.32 seconds
Epoch 2, Step 0, Loss: 1.5796875953674316
Epoch 2, Step 20, Loss: 1.6311328411102295
Epoch 2, Step 40, Loss: 1.6092958450317383
Epoch 2, Step 60, Loss: 1.7608165740966797
Epoch 2, Step 80, Loss: 1.1521098613739014
Epoch 2, Step 100, Loss: 1.4092659950256348
Epoch 2, Step 120, Loss: 1.2599318027496338
Epoch 2 completed in 29.11 seconds
Epoch 3, Step 0, Loss: 1.2696633338928223
Epoch 3, Step 20, Loss: 1.377254843711853
Epoch 3, Step 40, Loss: 1.1072807312011719
Epoch 3, Step 60, Loss: 1.297135829925537
Epoch 3, Step 80, Loss: 1.345583200454712
Epoch 3, Step 100, Loss: 1.0745306015014648
Epoch 3, Step 120, Loss: 1.0636987686157227
Epoch 3 completed in 30.15 second